In [1]:
from pathlib import Path
import json

I am mounting libcfgraph as a squashfs file with the udisks2 Arch package and the following commands
```bash
pv-squashfs-dev() { udisksctl loop-setup -f "$1" | grep -Eo '/dev/w+' ; }
pv-mount-dev() { udisksctl mount -b "$1" | grep -Eo '/run/media/(\w|/)+' ; }
pv-mount-link() { mkdir -p ~/mnt ; ln -s "$1" ~/mnt/"$2" ; }
pv-mount-squashfs() { pv-mount-link $(pv-mount-dev $(pv-squashfs-dev "$1")) "$2" ; }
```

I get the squashfs file by:
```bash
git clone --depth 1 https://github.com/regro/libcfgraph.git
mksquashfs ./libcfgraph ./libcfgraph.sfs -comp zstd -e .git
```
Note that mksquashfs is parallel and CPU bound, but this is 55GB is it'll take a while.

In [4]:
cfgraph_dir = Path("./mnt/libcfgraph")
list(cfgraph_dir.iterdir())

[PosixPath('mnt/libcfgraph/.circleci'),
 PosixPath('mnt/libcfgraph/.file_listing_0.json'),
 PosixPath('mnt/libcfgraph/.file_listing_1.json'),
 PosixPath('mnt/libcfgraph/.file_listing_2.json'),
 PosixPath('mnt/libcfgraph/.file_listing_3.json'),
 PosixPath('mnt/libcfgraph/.file_listing_4.json'),
 PosixPath('mnt/libcfgraph/.file_listing_5.json'),
 PosixPath('mnt/libcfgraph/.file_listing_6.json'),
 PosixPath('mnt/libcfgraph/.file_listing_7.json'),
 PosixPath('mnt/libcfgraph/.file_listing_8.json'),
 PosixPath('mnt/libcfgraph/.file_listing_9.json'),
 PosixPath('mnt/libcfgraph/.file_listing_meta.json'),
 PosixPath('mnt/libcfgraph/.github'),
 PosixPath('mnt/libcfgraph/.import_to_pkg_maps_indexed_files_0'),
 PosixPath('mnt/libcfgraph/.import_to_pkg_maps_indexed_files_1'),
 PosixPath('mnt/libcfgraph/.import_to_pkg_maps_indexed_files_2'),
 PosixPath('mnt/libcfgraph/.import_to_pkg_maps_indexed_files_3'),
 PosixPath('mnt/libcfgraph/.import_to_pkg_maps_indexed_files_4'),
 PosixPath('mnt/libcfgraph/.

In [10]:
next((cfgraph_dir / "artifacts/21cmfast/conda-forge/linux-64").iterdir())

PosixPath('mnt/libcfgraph/artifacts/21cmfast/conda-forge/linux-64/21cmfast-3.0.2-py36h1af98f8_1.json')

In [11]:
next((cfgraph_dir / "artifacts").glob("*/conda-forge/linux-64"))

PosixPath('mnt/libcfgraph/artifacts/21cmfast/conda-forge/linux-64')

In [24]:
with open("libcfgraph_reverse_index_linux-64.tsv","w") as out:
    for pkg_path in (cfgraph_dir / "artifacts").glob("*/conda-forge/linux-64"):
        # lexicographic sorting isn't the same as version sorting, but close enough for now
        json_path = sorted(pkg_path.glob("*.json"))[-1]
        with open(json_path) as f:
            meta = json.load(f)
        meta_name = meta['name']
        print("\n".join(("{}\t{}".format(file, meta_name) for file in meta['files'])), file=out)

Now you can browse it with `< libcfgraph_reverse_index_linux-64.tsv fzf`